# TSA Chapter 10: Granger Causality Heatmap

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_granger_heatmap/TSA_ch10_granger_heatmap.ipynb)

This notebook demonstrates:
- Testing pairwise Granger causality among macroeconomic variables and displaying results as a p-value heatmap.

In [ ]:
!pip install pandas-datareader statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
import pandas_datareader.data as web
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download and process macro data to quarterly frequency
try:
    series = {
        'GDP_growth': 'A191RL1Q225SBEA',
        'Unemployment': 'UNRATE',
        'Inflation': 'CPIAUCSL',
        'Fed_rate': 'FEDFUNDS',
    }
    dfs = {}
    for name, code in series.items():
        dfs[name] = web.DataReader(code, 'fred', '2000-01-01', '2025-01-01')
    gdp = dfs['GDP_growth'].resample('QS').last().dropna()
    unemp_q = dfs['Unemployment'].resample('QS').mean().dropna()
    cpi = dfs['Inflation'].resample('QS').last().dropna()
    cpi_yoy = cpi.pct_change(4) * 100
    fed = dfs['Fed_rate'].resample('QS').mean().dropna()
    macro = pd.DataFrame({
        'GDP_growth': gdp.iloc[:, 0], 'Unemployment': unemp_q.iloc[:, 0],
        'Inflation': cpi_yoy.iloc[:, 0], 'Fed_rate': fed.iloc[:, 0],
    }).dropna().loc['2001':]
    print(f'Macro data: {len(macro)} quarterly observations')
except Exception as e:
    print(f'FRED download failed: {e}. Using synthetic data.')
    np.random.seed(42)
    dates_q = pd.date_range('2001-01-01', '2025-01-01', freq='QS')
    n = len(dates_q)
    macro = pd.DataFrame({
        'GDP_growth': np.convolve(2.5 + np.random.normal(0, 1.5, n), np.ones(4)/4, mode='same'),
        'Unemployment': 5.5 + 1.5*np.sin(np.linspace(0, 4*np.pi, n)) + np.random.normal(0, 0.3, n),
        'Inflation': np.convolve(2.5 + np.random.normal(0, 0.8, n), np.ones(4)/4, mode='same'),
        'Fed_rate': np.maximum(2 + 2*np.sin(np.linspace(0, 3*np.pi, n)) + np.random.normal(0, 0.2, n), 0),
    }, index=dates_q)

# Granger causality tests
cols = macro.columns.tolist()
n_vars = len(cols)
pvals = np.ones((n_vars, n_vars))
max_lag = 4

for i, target in enumerate(cols):
    for j, cause in enumerate(cols):
        if i == j:
            pvals[i, j] = np.nan
            continue
        try:
            test_data = macro[[target, cause]].dropna()
            result = grangercausalitytests(test_data, maxlag=max_lag, verbose=False)
            min_p = min(result[lag][0]['ssr_ftest'][1] for lag in range(1, max_lag + 1))
            pvals[i, j] = min_p
        except Exception:
            pvals[i, j] = 1.0

fig, ax = plt.subplots(figsize=(6, 5))
masked = np.ma.array(pvals, mask=np.isnan(pvals))
im = ax.imshow(masked, cmap=plt.cm.RdYlGn_r, vmin=0, vmax=0.2, aspect='auto')
nice_labels = ['GDP growth', 'Unemployment', 'Inflation', 'Fed rate']
ax.set_xticks(range(n_vars)); ax.set_yticks(range(n_vars))
ax.set_xticklabels(nice_labels, rotation=30, ha='right')
ax.set_yticklabels(nice_labels)
ax.set_xlabel('Cause (column)'); ax.set_ylabel('Effect (row)')
ax.set_title('Granger Causality p-values')

for i in range(n_vars):
    for j in range(n_vars):
        if i == j:
            ax.text(j, i, '\u2014', ha='center', va='center', fontsize=9, color=COLORS['gray'])
        else:
            val = pvals[i, j]
            stars = '***' if val < 0.01 else ('**' if val < 0.05 else ('*' if val < 0.10 else ''))
            ax.text(j, i, f'{val:.3f}{stars}', ha='center', va='center', fontsize=7,
                    color='white' if val < 0.05 else 'black')
    ax.add_patch(plt.Rectangle((i-0.5, i-0.5), 1, 1, fill=True,
                                facecolor=COLORS['gray'], alpha=0.3, edgecolor='none'))

cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('p-value')
fig.tight_layout()
save_chart(fig, 'granger_heatmap')
plt.show()